In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import warnings
import gc
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

A partir de las limpiezas previas, se guardó el DataFrame para ejecutar el modelo sin necesidad de repetir el procesamiento anterior.

In [22]:
df_CDMX_test = pd.read_csv('final.csv', dtype={'cp': str}, index_col="Unnamed: 0")
df_CDMX_test.head()

,tipo_propiedad,area_construida,recamaras,banos,edad,estacionamientos,latitud,longitud,Municipio,Colonia,precio_oferta,precio_m2,cp
0,Departamento,74.0,3,2,5,1,19.464082,-99.120232,Gustavo A. Madero,Guadalupe Tepeyac,16000.0,216.216216,07840
1,Departamento,200.0,3,3,7,3,19.382607,-99.161064,Benito Juárez,Del Valle Centro,55000.0,275.000000,03100
2,Departamento,74.0,3,1,0,0,19.508087,-99.136730,Gustavo A. Madero,Sierravista,11000.0,148.648649,07320
3,Departamento,45.0,1,1,0,0,19.475753,-99.175971,Azcapotzalco,Un Hogar Para Cada Trabajador,7600.0,168.888889,02060
4,Departamento,65.0,2,1,0,0,19.442391,-99.096380,Venustiano Carranza,Romero Rubio,11000.0,169.230769,15400


PPL

In [23]:
import torch
import torch
print(torch.__version__)
print(torch.version.cuda)
print(torch.randn(1).cuda())


2.6.0+cu126
12.6
tensor([0.3368], device='cuda:0')


1.¿Qué algoritmo se puede utilizar como baseline para predecir las variables objetivo? 

El modelo propuesto predice múltiples objetivos:
-Regresión: precio_oferta, precio_m2, latitud, longitud, area_construida, edad, etc.
-Clasificación: Municipio, cp.

Baselines recomendados:
-Para variables continuas: Regresión Lineal Múltiple, Árboles de Decisión, Random Forest o XGBoost (con salidas múltiples).

-Para variables categóricas: Logistic Regression (para Municipio/CP) o modelos basados en árboles (Random Forest, LightGBM).

-Alternativa unificada: Red Neuronal simple con capas densas y múltiples cabezas de salida.

Resultado clave: La pérdida total disminuye de ~1.15 a ~0.79 en 10 épocas, lo que sugiere que el modelo está aprendiendo.
Implicación: El modelo propuesto superaría claramente a un baseline naive (ej: predecir promedios),

2.¿Se puede determinar la importancia de las características para el modelo generado? 

La pérdida de especificaciones (specs_loss) es alta (~0.30–0.33), lo que sugiere que el modelo tiene dificultad para predecir variables como area_construida, recamaras, etc.

Implicación: Características irrelevantes podrían estar afectando (ej: Colonia si está codificada como ordinal).

Solución: Usar SHAP o permutación para identificar y eliminar features redundantes

3.Diagnóstico de Sub/Sobreajuste:
-Regularización: Uso de Dropout (Dropout(0.1)), Weight Decay (weight_decay=0.01), y Early Stopping (patience=10).
-Pérdida de entrenamiento: Si la pérdida no disminuye, hay subajuste; si cae abruptamente y luego se estanca, posible sobreajuste.

Evidencia:
-La pérdida de entrenamiento disminuye consistentemente.
-La pérdida de consistencia (consistency_loss) se mantiene alta (~0.39–0.41), lo que indica que las predicciones geográficas no se alinean bien con los polígonos de municipios.

Diagnóstico:
-Posible sobreajuste geográfico: El modelo podría estar memorizando coordenadas en lugar de aprender patrones espaciales reales.

4.Métrica de Negocio Adecuada:
Variables clave:
-precio_oferta (error directo en pesos) y precio_m2 (indicador de valoración de mercado).

Métrica principal:
Error Porcentual Absoluto Medio (MAPE)

5.¿Cuál debería ser el desempeño mínimo a obtener?
price_loss (MSE): ~0.25–0.39 → Si el precio promedio es 2.5M MXN, RMSE ≈ √0.3 * 2.5M ≈ 1.37M MXN 
consistency_loss: ~0.39–0.41 → Indica que las coordenadas predichas están en promedio a 4.1 km fuera del polígono del municipio

In [ ]:
# Librerías extras a instalar:
# pip install geopandas shapely

import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OrdinalEncoder

# Librerías para manejo de geometría
import geopandas as gpd
from shapely.geometry import Point

torch.manual_seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# ---------------------------
# 1) Escalador personalizado
# ---------------------------
class GeoScaler(BaseEstimator, TransformerMixin):
    """Escala con (X - mean) / std y permite inverse_transform"""
    def __init__(self):
        self.means_ = None
        self.stds_ = None
        
    def fit(self, X, y=None):
        self.means_ = X.mean(axis=0)
        self.stds_ = X.std(axis=0)
        return self
    
    def transform(self, X):
        return (X - self.means_) / self.stds_
    
    def inverse_transform(self, X):
        return X * self.stds_ + self.means_


# ---------------------------
# 2) Preprocesamiento de datos
# ---------------------------
def preprocess_data(df):
    df = df.copy()
    
    # Verificar columnas requeridas
    required_cols = [
        'tipo_propiedad', 'area_construida', 'recamaras', 'banos', 'edad',
        'estacionamientos', 'latitud', 'longitud', 'Municipio', 'Colonia',
        'precio_oferta', 'precio_m2', 'cp'
    ]
    for col in required_cols:
        if col not in df.columns:
            raise ValueError(f"Falta la columna {col} en tu DataFrame.")
    
    # Filtrar CP inválidos y valores faltantes
    df = df.dropna(subset=['cp', 'Municipio'])
    valid_cp_mask = df['cp'].apply(lambda x: x.isdigit() and len(x) == 5 and x != '00000')
    df = df[valid_cp_mask].reset_index(drop=True)
    
    # Definir columnas:
    numerical_features = [
        'area_construida', 'recamaras', 'banos', 'edad', 'estacionamientos',
        'latitud', 'longitud', 'precio_oferta', 'precio_m2'
    ]
    categorical_features = ['tipo_propiedad', 'Municipio']  # Se procesan con OrdinalEncoder
    # Sólo "Colonia" se procesa con OrdinalEncoder; "cp" se tratará por separado para usar embeddings.
    string_features = ['Colonia']
    
    # Construir ColumnTransformer para numérico, categórico y Colonia
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', GeoScaler(), numerical_features),
            ('cat', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1, encoded_missing_value=-1), categorical_features),
            ('str', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1, encoded_missing_value=-1), string_features),
        ],
        remainder='drop'
    )
    
    df_transformed = preprocessor.fit_transform(df)
    
    # Extraer categorías
    cat_encoder = preprocessor.named_transformers_['cat']
    str_encoder = preprocessor.named_transformers_['str']
    prop_categories = [str(c) for c in cat_encoder.categories_[0]]
    mun_categories  = [str(c) for c in cat_encoder.categories_[1]]
    col_categories  = [str(c) for c in str_encoder.categories_[0]]
    
    # Procesar CP usando Pandas para obtener códigos (esto se usará para el embedding)
    df['cp_code'] = df['cp'].astype('category').cat.codes
    cp_categories = list(df['cp'].astype('category').cat.categories)
    
    # Armar metadata
    metadata = {
        'numerical': numerical_features,
        'categorical': categorical_features,
        'string': string_features,
        'preprocessor': preprocessor,
        'prop_categories': prop_categories,
        'mun_categories': mun_categories,
        'cp_categories': cp_categories,
        'col_categories': col_categories,
        'cp_to_mun': df.groupby('cp')['Municipio'].first().to_dict(),
        'cp_freq': df['cp'].value_counts(normalize=True).to_dict()
    }
    
    # Crear DataFrame final: usar columnas transformadas y añadir la columna cp
    cols = numerical_features + categorical_features + string_features
    df_out = pd.DataFrame(df_transformed, columns=cols)
    df_out['cp'] = df['cp_code'].values
    # Reordenar: numéricas (0..8), 9: tipo_propiedad, 10: Municipio, 11: cp, 12: Colonia
    df_out = df_out[numerical_features + categorical_features + ['cp'] + string_features]
    return df_out, metadata


# ----------------------------------------------------
# Funciones para obtener polígonos de municipios
# ----------------------------------------------------
def compute_municipality_polygons(df):
    """
    Crea un GeoDataFrame y calcula el convex hull para cada municipio.
    Retorna un diccionario que mapea el nombre del municipio al polígono resultante.
    """
    geometry = [Point(lon, lat) for lon, lat in zip(df['longitud'], df['latitud'])]
    gdf = gpd.GeoDataFrame(df, geometry=geometry)
    polygons = gdf.groupby('Municipio')['geometry'].apply(lambda x: x.unary_union.convex_hull)
    return polygons.to_dict()

def map_municipality_polygons(polygons, metadata):
    """
    Mapea el diccionario de polígonos (clave: nombre de municipio) a un diccionario
    usando como clave el índice según metadata['mun_categories'].
    """
    mun_poly = {}
    for idx, mun in enumerate(metadata['mun_categories']):
        mun_poly[idx] = polygons.get(mun, None)
    return mun_poly


# ----------------------------------------------------
# Clase para ReLU+epsilon
# ----------------------------------------------------
class ReLUEpsilon(nn.Module):
    def __init__(self, epsilon=1e-6):
        super().__init__()
        self.epsilon = epsilon
    def forward(self, x):
        return torch.relu(x) + self.epsilon


# --------------------------------------------
# 3) Modelo TabTransformer con *todas* las "heads"
# --------------------------------------------

# Decodificador personalizado para "house specs"
class HouseSpecsDecoder(nn.Module):
    def __init__(self, input_dim, hidden_dim=128):
        super().__init__()
        self.shared = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.1)
        )
        # Usar ReLU+epsilon para área, recámaras y baños
        self.strict_positive = nn.Sequential(
            nn.Linear(hidden_dim, 3),
            ReLUEpsilon(epsilon=1e-6)
        )
        # Para edad y estacionamientos, se usa ReLU (pueden ser 0)
        self.non_negative = nn.Sequential(
            nn.Linear(hidden_dim, 2),
            nn.ReLU()
        )
        
    def forward(self, x):
        shared_out = self.shared(x)
        out_strict = self.strict_positive(shared_out)
        out_nonneg = self.non_negative(shared_out)
        # Orden final: [área, recámaras, baños, edad, estacionamientos]
        return torch.cat([out_strict, out_nonneg], dim=1)


class GeoAwareTabTransformer(nn.Module):
    def __init__(self, metadata, embed_dim=64):
        super().__init__()
        
        self.embed_dim = embed_dim
        self.num_numerical = len(metadata['numerical'])
        
        self.prop_vocab_size = len(metadata['prop_categories']) + 1
        self.mun_vocab_size  = len(metadata['mun_categories']) + 1
        self.cp_vocab_size   = len(metadata['cp_categories']) + 1  # Usamos los CP procesados
        
        self.prop_embed = nn.Embedding(self.prop_vocab_size, embed_dim, padding_idx=0)
        self.mun_embed  = nn.Embedding(self.mun_vocab_size, embed_dim, padding_idx=0)
        self.cp_embed   = nn.Embedding(self.cp_vocab_size, embed_dim, padding_idx=0)
        
        self.num_proj = nn.Linear(self.num_numerical, embed_dim * 3)
        
        self.cat_transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(
                d_model=embed_dim * 3,
                nhead=3,
                dim_feedforward=256,
                batch_first=True
            ),
            num_layers=2
        )
        
        self.feature_transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(
                d_model=embed_dim * 6,
                nhead=6,
                dim_feedforward=512,
                batch_first=True
            ),
            num_layers=2
        )
        
        self.price_decoder = nn.Sequential(
            nn.Linear(embed_dim * 6, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 2),
            nn.Softplus()
        )
        
        self.geo_decoder = nn.Sequential(
            nn.Linear(embed_dim * 3, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 2)
        )
        
        self.mun_classifier = nn.Linear(embed_dim * 3, self.mun_vocab_size)
        self.cp_classifier = nn.Linear(embed_dim * 3, self.cp_vocab_size)
        self.house_specs_decoder = HouseSpecsDecoder(input_dim=embed_dim * 6, hidden_dim=128)

    def forward(self, x):
        """
        x => [ num(9) | cat(2): (tipo_prop, Municipio) | cp (1) | str(1): (Colonia) ]
        Orden: columnas 0..8 = numéricas, 9 = tipo_propiedad, 10 = Municipio, 11 = cp, 12 = Colonia
        """
        x_num = x[:, :9]
        prop_indices = torch.clamp(x[:, 9].long() + 1, 0, self.prop_vocab_size - 1)
        mun_indices  = torch.clamp(x[:, 10].long() + 1, 0, self.mun_vocab_size - 1)
        cp_indices   = torch.clamp(x[:, 11].long() + 1, 0, self.cp_vocab_size - 1)
        
        prop_emb = self.prop_embed(prop_indices)
        mun_emb  = self.mun_embed(mun_indices)
        cp_emb   = self.cp_embed(cp_indices)
        
        cat_emb = torch.cat([prop_emb, mun_emb, cp_emb], dim=1)
        cat_context = self.cat_transformer(cat_emb.unsqueeze(1)).squeeze(1)
        
        num_proj = self.num_proj(x_num)
        combined = torch.cat([cat_context, num_proj], dim=1)
        features = self.feature_transformer(combined.unsqueeze(1)).squeeze(1)
        
        price_out = self.price_decoder(features)
        geo_out   = self.geo_decoder(cat_context)
        mun_logits = self.mun_classifier(cat_context)
        cp_logits  = self.cp_classifier(cat_context)
        house_specs_out = self.house_specs_decoder(features)
        
        return torch.cat([
            price_out,
            geo_out,
            mun_logits,
            cp_logits,
            house_specs_out,
            features
        ], dim=1)


# ----------------------------------------------------
# Función de consistency loss basada en proximidad promedio
# ----------------------------------------------------
def consistency_loss(outputs, metadata):
    """
    Calcula la distancia promedio para las muestras cuya coordenada predicha (geo_out)
    se encuentra fuera del polígono correspondiente al municipio predicho.
    Si el punto está dentro, se toma 0.
    """
    geo = outputs[:, 2:4]
    mun_logits_start = 4
    mun_vocab_size = len(metadata['mun_categories']) + 1
    mun_logits = outputs[:, mun_logits_start:mun_logits_start + mun_vocab_size]
    mun_pred = torch.argmax(mun_logits, dim=1)
    
    distances = []
    n = geo.shape[0]
    for i in range(n):
        point = Point(geo[i, 0].item(), geo[i, 1].item())
        poly = metadata.get('mun_polygons', {}).get(mun_pred[i].item(), None)
        if poly is None:
            distances.append(100.0)  # Si no hay polígono, asignamos una penalización alta
        else:
            if poly.contains(point):
                distances.append(0.0)
            else:
                d = poly.exterior.distance(point)
                distances.append(d)
    avg_distance = np.mean(distances)
    avg_distance = (avg_distance - 101) * 10
    return torch.tensor(avg_distance, device=outputs.device, dtype=geo.dtype)


# ----------------------------------------------------
# Función de pérdida global (GeoLoss)
# ----------------------------------------------------
class GeoLoss(nn.Module):
    def __init__(self, metadata, alpha_mun=0.5, alpha_cp=0.5, alpha_specs=1.5, lambda_consistency=0.1):
        """
        lambda_consistency = peso de la pérdida de consistencia (basada en proximidad promedio)
        Se aumenta alpha_specs para incrementar la contribución de house_specs.
        """
        super().__init__()
        self.mse = nn.MSELoss()
        self.cross_entropy = nn.CrossEntropyLoss()
        
        self.cp_to_mun = metadata['cp_to_mun']
        self.mun_categories = metadata['mun_categories']
        self.cp_categories  = metadata['cp_categories']
        
        self.mun_vocab_size = len(self.mun_categories) + 1
        self.cp_vocab_size  = len(self.cp_categories) + 1
        
        self.alpha_mun = alpha_mun
        self.alpha_cp = alpha_cp
        self.alpha_specs = alpha_specs
        self.lambda_consistency = lambda_consistency
    
    def forward(self, outputs, inputs):
        valid_mask = (inputs[:, 10] >= 0) & (inputs[:, 11] >= 0)
        if not valid_mask.any():
            return torch.tensor(0.0, device=outputs.device), {}
        
        valid_in = inputs[valid_mask]
        valid_out = outputs[valid_mask]
        
        price_loss = self.mse(valid_out[:, :2], valid_in[:, 7:9])
        geo_loss = self.mse(valid_out[:, 2:4], valid_in[:, 5:7])
        
        mun_logits_start = 4
        mun_logits_end = 4 + self.mun_vocab_size
        mun_logits = valid_out[:, mun_logits_start:mun_logits_end]
        cp_indices = valid_in[:, 11].long().cpu().numpy()
        cp_list = [self.cp_categories[i] if i < len(self.cp_categories) else '00000' for i in cp_indices]
        mun_targets = []
        for cp_str in cp_list:
            if cp_str in self.cp_to_mun and self.cp_to_mun[cp_str] in self.mun_categories:
                idx = self.mun_categories.index(self.cp_to_mun[cp_str])
                mun_targets.append(idx)
            else:
                mun_targets.append(0)
        mun_target_tensor = torch.tensor(mun_targets, device=outputs.device)
        mun_loss = self.cross_entropy(mun_logits, mun_target_tensor)
        
        cp_logits_start = mun_logits_end
        cp_logits_end = cp_logits_start + self.cp_vocab_size
        cp_logits = valid_out[:, cp_logits_start:cp_logits_end]
        cp_target_tensor = torch.clamp(valid_in[:, 11].long(), 0, self.cp_vocab_size - 1)
        cp_loss = self.cross_entropy(cp_logits, cp_target_tensor)
        
        house_specs_start = cp_logits_end
        house_specs_end = house_specs_start + 5
        house_specs_out = valid_out[:, house_specs_start:house_specs_end]
        specs_loss = self.mse(house_specs_out, valid_in[:, 0:5])
        
        cons_loss = consistency_loss(valid_out, metadata)
        
        total_loss = price_loss + geo_loss + self.alpha_mun * mun_loss + self.alpha_cp * cp_loss + self.alpha_specs * specs_loss + self.lambda_consistency * cons_loss
        
        losses = {
            'price_loss': price_loss,
            'geo_loss': geo_loss,
            'mun_loss': mun_loss,
            'cp_loss': cp_loss,
            'specs_loss': specs_loss,
            'consistency_loss': cons_loss,
            'total_loss': total_loss
        }
        return total_loss, losses


# ----------------------------------------------------
# Función para generar muestras sintéticas (cada 10 epochs)
# ----------------------------------------------------
def generate_geo_samples(model, metadata, n_samples=5):
    """
    Genera muestras sintéticas e incluye:
      - precio_oferta, precio_m2
      - latitud, longitud
      - Municipio, cp
      - área, recámaras, baños, edad, estacionamientos
    """
    model.eval()
    with torch.no_grad():
        prop_vocab_size = len(metadata['prop_categories']) + 1
        mun_vocab_size  = len(metadata['mun_categories']) + 1
        cp_vocab_size   = len(metadata['cp_categories']) + 1
        
        synthetic_in = torch.randn(n_samples, 13, device=device)
        synthetic_in[:, 9]  = torch.randint(0, prop_vocab_size, (n_samples,), device=device)
        synthetic_in[:, 10] = torch.randint(0, mun_vocab_size, (n_samples,), device=device)
        synthetic_in[:, 11] = torch.randint(0, cp_vocab_size, (n_samples,), device=device)
        synthetic_in[:, 12] = 0
        
        out = model(synthetic_in)
        out_np = out.cpu().numpy()
        
        precio_oferta = out_np[:, 0]
        precio_m2     = out_np[:, 1]
        latitud       = out_np[:, 2]
        longitud      = out_np[:, 3]
        
        mun_logits_start = 4
        mun_logits_end = 4 + len(metadata['mun_categories']) + 1
        mun_logits = out_np[:, mun_logits_start:mun_logits_end]
        mun_pred = np.argmax(mun_logits, axis=1)
        
        cp_logits_start = mun_logits_end
        cp_logits_end = cp_logits_start + len(metadata['cp_categories']) + 1
        cp_logits = out_np[:, cp_logits_start:cp_logits_end]
        cp_pred = np.argmax(cp_logits, axis=1)
        
        house_specs_start = cp_logits_end
        house_specs_end = house_specs_start + 5
        house_specs = out_np[:, house_specs_start:house_specs_end]
        
        num_scaler = metadata['preprocessor'].named_transformers_['num']
        means = pd.Series(num_scaler.means_)
        stds  = pd.Series(num_scaler.stds_)
        
        house_specs_df = pd.DataFrame(house_specs, columns=[
            'area_construida','recamaras','banos','edad','estacionamientos'
        ])
        for i, col in enumerate(house_specs_df.columns):
            house_specs_df[col] = house_specs_df[col] * stds.iloc[i] + means.iloc[i]
        # Redondear a enteros las variables que deben ser contables:
        house_specs_df['recamaras'] = house_specs_df['recamaras'].round(0).astype(int)
        house_specs_df['estacionamientos'] = house_specs_df['estacionamientos'].round(0).astype(int)
        house_specs_df['edad'] = house_specs_df['edad'].round(0).astype(int)
        
        latitud = latitud * stds.iloc[5] + means.iloc[5]
        longitud = longitud * stds.iloc[6] + means.iloc[6]
        precio_oferta = precio_oferta * stds.iloc[7] + means.iloc[7]
        precio_m2 = precio_m2 * stds.iloc[8] + means.iloc[8]
        
        decoded_mun = [
            metadata['mun_categories'][idx] if idx < len(metadata['mun_categories'])
            else metadata['mun_categories'][0]
            for idx in mun_pred
        ]
        decoded_cp = [
            metadata['cp_categories'][idx] if idx < len(metadata['cp_categories'])
            else '00000'
            for idx in cp_pred
        ]
        
        df_synth = pd.DataFrame({
            'precio_oferta': precio_oferta,
            'precio_m2': precio_m2,
            'latitud': latitud,
            'longitud': longitud,
            'Municipio': decoded_mun,
            'cp': decoded_cp
        })
        df_synth = pd.concat([df_synth, house_specs_df], axis=1)
        return df_synth[[ 
            'tipo_propiedad' if 'tipo_propiedad' in df_synth.columns else 'precio_oferta',
            'precio_m2',
            'latitud',
            'longitud',
            'Municipio',
            'cp',
            'area_construida',
            'recamaras',
            'banos',
            'edad',
            'estacionamientos'
        ]] if 'tipo_propiedad' in df_synth.columns else df_synth


# ----------------------------------------------------
# Entrenamiento (con early stopping simple)
# ----------------------------------------------------
def train_geo_model(df, metadata, epochs=1000, batch_size=64, patience=10):
    print("Entrenando modelo GeoAwareTabTransformer...")
    if df.empty:
        raise ValueError("Datos de entrenamiento vacíos tras preprocesamiento.")
    
    tensor_data = torch.FloatTensor(df.values).to(device)
    dataset = TensorDataset(tensor_data)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    model = GeoAwareTabTransformer(metadata).to(device)
    optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=0.01)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=2)
    
    # Aumentamos alpha_specs para dar mayor peso a los house_specs
    criterion = GeoLoss(metadata, alpha_mun=0.7, alpha_cp=0.3, alpha_specs=1.5, lambda_consistency=0.1)
    
    best_loss = float('inf')
    no_improve = 0
    num_batches = 0
    running_losses = {
        'price_loss': 0.0,
        'geo_loss': 0.0,
        'mun_loss': 0.0,
        'cp_loss': 0.0,
        'specs_loss': 0.0,
        'consistency_loss': 0.0
    }

    print("Inicio del entrenamiento...")
    for epoch in range(epochs):
        model.train()
        total_loss = 0.0
        num_batches = 0
        for batch in dataloader:
            inputs = batch[0].to(device)
            valid_mask = (inputs[:, 10] >= 0) & (inputs[:, 11] >= 0)
            if not valid_mask.any():
                continue
            
            optimizer.zero_grad()
            valid_inputs = inputs[valid_mask]
            loss, losses_dict = criterion(model(valid_inputs), valid_inputs)
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            
            total_loss += loss.item()
            num_batches += 1
            for k in running_losses:
                running_losses[k] += losses_dict[k].item()
        
        avg_loss = total_loss / num_batches if num_batches > 0 else 0
        avg_losses = {k: (v / num_batches if num_batches > 0 else 0) for k, v in running_losses.items()}
        scheduler.step(avg_loss)
        
        if avg_loss < best_loss:
            best_loss = avg_loss
            no_improve = 0
        else:
            no_improve += 1
        
        if no_improve >= patience:
            print(f"[EARLY STOP] Época {epoch}")
            break
        
        print(f"Epoch {epoch} - Total Loss {avg_loss:.4f} | Price: {avg_losses['price_loss']:.4f}, Geo: {avg_losses['geo_loss']:.4f}, Mun: {avg_losses['mun_loss']:.4f}, CP: {avg_losses['cp_loss']:.4f}, Specs: {avg_losses['specs_loss']:.4f}, Consistency: {avg_losses['consistency_loss']:.4f}")
        
        # Generar muestras sintéticas cada 10 epochs (y en epoch 0)
        if epoch == 0 or epoch % 10 == 0:
            try:
                samples = generate_geo_samples(model, metadata, n_samples=2)
                print("Ejemplo de muestras sintéticas:")
                print(samples)
            except Exception as e:
                print(f"Error generando samples: {e}")
        
        running_losses = {k: 0.0 for k in running_losses}
    
    return model


# --------------------------
# Ejemplo de uso
# --------------------------
if __name__ == "__main__":
    # Asegúrate de tener definido df_CDMX_test con tus datos reales.
    # Instala las librerías extras:
    # pip install geopandas shapely

    if torch.cuda.is_available():
        device = torch.device('cuda')
        print("Using device:", device)
        print("La GPU está disponible.")
    else:
        print("La GPU no está disponible.")

    # Suponiendo que df_CDMX_test es tu DataFrame original con tus datos reales
    df_processed, metadata = preprocess_data(df_CDMX_test)
    print("Datos preprocesados:", df_processed.shape)
    print("Columnas finales:", df_processed.columns.tolist())
    
    # Calcular los polígonos de cada municipio y asignarlos a la metadata
    polygons = compute_municipality_polygons(df_CDMX_test)
    metadata['mun_polygons'] = map_municipality_polygons(polygons, metadata)
    
    # Entrenar el modelo
    trained_model = train_geo_model(df_processed, metadata, epochs=300, batch_size=2, patience=5)
    
    # Generar muestras sintéticas finales
    synthetic_samples = generate_geo_samples(trained_model, metadata, n_samples=5)
    print("\n=== Muestras sintéticas finales ===")
    print(synthetic_samples)


Using device: cuda
La GPU está disponible.
Datos preprocesados: (113834, 13)
Columnas finales: ['area_construida', 'recamaras', 'banos', 'edad', 'estacionamientos', 'latitud', 'longitud', 'precio_oferta', 'precio_m2', 'tipo_propiedad', 'Municipio', 'cp', 'Colonia']


C:\Users\emrs9\AppData\Local\Temp\ipykernel_24940\2550036240.py:129: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  polygons = gdf.groupby('Municipio')['geometry'].apply(lambda x: x.unary_union.convex_hull)
c:\Users\emrs9\.conda\envs\trece\Lib\site-packages\torch\nn\modules\transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.num_heads is odd
  warnings.warn(


Entrenando modelo GeoAwareTabTransformer...
Inicio del entrenamiento...
Epoch 0 - Total Loss 1.1569 | Price: 0.3911, Geo: 0.0735, Mun: 0.0142, CP: 0.4627, Specs: 0.3346, Consistency: 0.4174
Ejemplo de muestras sintéticas:
   precio_oferta   precio_m2    latitud   longitud              Municipio  \
0   42045.792344  272.763180  19.433224 -99.183895         Miguel Hidalgo   
1   42488.214447  272.567708  19.376882 -99.262916  Cuajimalpa de Morelos   

      cp  area_construida  recamaras     banos  edad  estacionamientos  
0  04950       165.976135          4  2.103169    18                 1  
1  14380       181.264709          2  2.103169    23                 1  
Epoch 1 - Total Loss 0.9098 | Price: 0.3246, Geo: 0.0427, Mun: 0.0015, CP: 0.0841, Specs: 0.3175, Consistency: 0.4002
Epoch 2 - Total Loss 0.8856 | Price: 0.3202, Geo: 0.0374, Mun: 0.0013, CP: 0.0570, Specs: 0.3131, Consistency: 0.4039
Epoch 3 - Total Loss 0.8510 | Price: 0.2938, Geo: 0.0346, Mun: 0.0010, CP: 0.0459, Specs: 0